## Génération de résumé

L'objectif de ce notebook est de comparer les performances de génération de résumé entre un modèle baseline et le modèle T5 (Text-to-Text Transfer Transformer de Google Research). 

### Dataset
- Présentation de CNN /Daily Mail
- Exploration aléatoire du dataset
### Modèle baseline: TextRank
- Génération des résumés
- Score Rouge
### Modèle T5
- Préparation des données
- Prétraitement des données
- Fine-tuning du modèle
- Génération de résumés et calcul du score Rouge
- Optimisation du modèle: changement du nombre d'epochs
- Sauvegarde des articles et résumés générés dans un fichier CSV


## Dataset

### Présentation de CNN/Daily Mail
On utilise le dataset CNN/Daily Mail fourni par Hugging Face.  
Le CNN/Daily Mail dataset est largement utilisé dans la communauté de recherche en NLP pour évaluer les performances des algorithmes de résumé automatique.  

Chaque entrée dans le dataset contient deux parties principales :  
Article : l'article complet provenant de CNN ou de Daily Mail. Mélange d'informations factuelles, de descriptions et de commentaires.  
Highlights: les résumés "Highlights" fournissent une référence de qualité pour évaluer les résumés générés par des modèles automatiques.  
Pour éviter des calculs trop longs, on se limitera à environ 130 lignes du dataset.

In [16]:
from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0', split='validation[:1%]')

In [17]:
dataset.shape

(134, 3)

In [18]:
csv_file_path = 'cnn_dailymail_origine.csv'
dataset.to_csv(csv_file_path)

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 29.57ba/s]


491882

### Exploration aléatoire du dataset

In [ ]:
import random
sample_indices = random.sample(range(len(dataset)), 5)  # génère 5 indices uniques
sampled_dataset = [dataset[i] for i in sample_indices]  # récupère les échantillons correspondants

for article in sampled_dataset:
    print(article)


{'article': '(CNN)A U.S. Air Force veteran who allegedly tried to join ISIS in Syria but was turned back by Turkish authorities before he could get to the war-torn country entered a not guilty plea to terror-related charges Wednesday in a federal court in New York. Tairod Nathan Webster Pugh, accused of making the foiled attempt in January, was indicted by a grand jury on charges of trying to give material support to the terror group and obstruction of justice, the U.S. Justice Department said in a two-count indictment announced Tuesday. Among the evidence, prosecutors allege: Investigators discovered on his laptop computer a letter saying he wanted to "use the talents and skills given to me by Allah to establish and defend the Islamic States," and a chart of crossing points between Turkey and Syria, where ISIS controls some territory. Who has been recruited to ISIS from the West? Pugh, a 47-year-old convert to Islam and a former New Jersey resident who served in the Air Force from 198

## Modèle baseline: TextRank

TextRank est une méthode d’extraction de texte basée sur des algorithmes de classement de graphes.

**Principe de TextRank**: des phrases dans un texte peuvent être représentées comme des nœuds dans un graphe, et les similitudes entre différentes phrases peuvent être représentées comme des arêtes reliant ces nœuds.

### Génération des résumés avec TextRank

In [4]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

def summarize_textrank(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, 3)
    return " ".join([str(sentence) for sentence in summary])

baseline_summaries = [summarize_textrank(article['article']) for article in dataset]

### Calcul du score Rouge

Le score Rouge (Recall-Oriented Understudy for Gisting Evaluation) est une mesure standard pour évaluer la qualité des résumés générés par rapport à des résumés de référence.  
Le score Rouge-1 mesure le chevauchement des unigrammes (mots individuels) entre le résumé généré et le résumé de référence. Le score Rouge-2 est l'équivalent pour les bigrammes (paires de mots). Le score Rouge-L est l'équivalent pour les plus longues séquences.

In [5]:
from rouge import Rouge
import numpy as np

rouge = Rouge()

rouge_scores_baseline = [rouge.get_scores(baseline_summary, article['highlights']) for baseline_summary, article in zip(baseline_summaries, dataset)]

print("Score ROUGE moyen pour TextRank (baseline) :", np.mean([score[0]['rouge-1']['f'] for score in rouge_scores_baseline]))

Score ROUGE moyen pour TextRank (baseline) : 0.20203824760177294


## Modèle T5  

Text-to-Text Transfer Transformer : architecture de modèle NLP développée par Google Research.  
L’architecture suit la structure encodeur / décodeur.  
L'encodeur lit et encode le texte d'entrée en vecteurs de caractéristiques.  
Le décodeur utilise les vecteurs de caractéristiques pour générer le texte de sortie, mot par mot.

### Préparation des données

Division du dataset en ensembles d'entraînement, de validation et de test.

In [ ]:
# Définition des proportions pour le split
train_size = 0.8
val_size = 0.1  # et test_size sera de 0.1 également

# Calcul des indices de coupe pour le dataset
train_split = int(len(dataset) * train_size)
val_split = train_split + int(len(dataset) * val_size)

train_dataset = dataset.select(range(0, train_split))
val_dataset = dataset.select(range(train_split, val_split))
test_dataset = dataset.select(range(val_split, len(dataset)))

# Taille de chaque split
print("Taille du dataset d'entraînement :", len(train_dataset))
print("Taille du dataset de validation :", len(val_dataset))
print("Taille du dataset de test :", len(test_dataset))


Taille du dataset d'entraînement : 107
Taille du dataset de validation : 13
Taille du dataset de test : 14


### Prétraitement des données

On prétraite les données pour le formatage requis par T5. Cela inclut l'ajout du préfixe "summarize:" devant chaque texte et la conversion des données au format approprié.

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def preprocess_data(batch):
    # prétraitement de chaque article et highlight dans le lot
    input_texts = ['summarize: ' + article for article in batch['article']]
    input_encodings = tokenizer(input_texts, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    label_encodings = tokenizer(batch['highlights'], padding='max_length', truncation=True, max_length=150, return_tensors='pt')

    # on s'assure que 'input_ids' et 'labels' sont des listes de ids
    batch['input_ids'] = input_encodings.input_ids
    batch['labels'] = label_encodings.input_ids

    return batch


train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Fine-tuning du modèle

Configuration et fine-tuning du modèle T5 sur l'ensemble de données d'entraînement.

Des techniques d'optimisation sont employées pour améliorer la convergence du modèle.

1. **Warmup Linéaire** : L'argument `warmup_steps=500` dans `TrainingArguments` indique que le warmup linéaire est utilisé. Pendant les 500 premières étapes d'entraînement, le taux d'apprentissage augmente linéairement depuis zéro jusqu'au taux d'apprentissage initial défini (non spécifié explicitement, donc il utilise la valeur par défaut du `Trainer`). Cette technique aide à stabiliser l'entraînement au début, en évitant les mises à jour trop importantes qui pourraient nuire à la convergence initiale du modèle.

2. **Décroissance du Poids (Weight Decay)** : L'argument `weight_decay=0.01` signifie qu'une pénalité est ajoutée à la perte, proportionnelle aux poids du modèle, encourageant ainsi le modèle à apprendre des poids plus petits (en valeur absolue). Cette technique est utilisée pour régulariser le modèle et prévenir le surajustement, ce qui peut conduire à une meilleure généralisation sur des données non vues.

Le warmup linéaire assure que l'entraînement démarre doucement, réduisant le risque de perturbations importantes dues à des mises à jour trop agressives au début de l'entraînement. La décroissance du poids encourage le modèle à rester simple, ce qui peut améliorer la capacité du modèle à généraliser à partir de l'ensemble d'entraînement à l'ensemble de validation ou de test, améliorant ainsi la performance globale du modèle sur des données nouvelles ou non vues.

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


### Génération de résumés et calcul des scores ROUGE

Après le fine-tuning, on utilise le modèle pour générer des résumés sur l'ensemble de test et calculer les scores ROUGE.

In [9]:
from rouge import Rouge

rouge = Rouge()
generated_summaries = []
references = []

for batch in test_dataset:
    input_ids = tokenizer('summarize: ' + batch['article'], return_tensors='pt', truncation=True, padding='max_length', max_length=512).input_ids
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(generated_summary)
    references.append(batch['highlights'])

rouge_scores = rouge.get_scores(generated_summaries, references, avg=True)
print(rouge_scores)

{'rouge-1': {'r': 0.38149605386052954, 'p': 0.30319058746522215, 'f': 0.32774647502072046}, 'rouge-2': {'r': 0.17753578028822428, 'p': 0.13766313299248464, 'f': 0.15022297710984672}, 'rouge-l': {'r': 0.3415781449720323, 'p': 0.2701278099171505, 'f': 0.29195519074747683}}


In [10]:
import numpy as np
# scores ROUGE pour chaque paire de résumé généré et de référence
rouge_scores_finetuned = [rouge.get_scores(gen_summary, ref) for gen_summary, ref in zip(generated_summaries, references)]

# moyenne des scores F1 de ROUGE-1 pour tous les résumés
average_rouge1_f_score_finetuned = np.mean([score[0]['rouge-1']['f'] for score in rouge_scores_finetuned])
print("Score ROUGE moyen pour T5 avec entraînement :", average_rouge1_f_score_finetuned)

Score ROUGE moyen pour T5 avec entraînement : 0.32774647502072046


### Optimisation du modèle: changement du nombre d'epochs

On modifie le nombre d'epochs pour voir si cela peut augmenter les performances du modèle T5 en terme de score Rouge.  
Pour ce faire on crée un nouveau modèle t5-small avec un num_train_epochs égal à 4.

In [11]:
model_epochs = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model_epochs,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


  9%|▉         | 10/108 [00:41<06:53,  4.22s/it]

{'loss': 9.6083, 'grad_norm': 50.46287536621094, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.37}


 19%|█▊        | 20/108 [01:23<06:07,  4.18s/it]

{'loss': 9.5346, 'grad_norm': 38.754150390625, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.74}


 25%|██▌       | 27/108 [01:59<06:13,  4.62s/it]

...

 93%|█████████▎| 100/108 [07:29<00:32,  4.03s/it]

{'loss': 6.0439, 'grad_norm': 37.55124282836914, 'learning_rate': 1e-05, 'epoch': 3.7}


100%|██████████| 108/108 [08:06<00:00,  4.51s/it]

{'eval_loss': 6.074146747589111, 'eval_runtime': 3.458, 'eval_samples_per_second': 3.759, 'eval_steps_per_second': 1.157, 'epoch': 4.0}
{'train_runtime': 486.7038, 'train_samples_per_second': 0.879, 'train_steps_per_second': 0.222, 'train_loss': 8.060311705977828, 'epoch': 4.0}


TrainOutput(global_step=108, training_loss=8.060311705977828, metrics={'train_runtime': 486.7038, 'train_samples_per_second': 0.879, 'train_steps_per_second': 0.222, 'train_loss': 8.060311705977828, 'epoch': 4.0})

In [12]:
rouge = Rouge()
generated_epochs_summaries = []
references = []

for batch in test_dataset:
    input_ids = tokenizer('summarize: ' + batch['article'], return_tensors='pt', truncation=True, padding='max_length', max_length=512).input_ids
    summary_ids = model_epochs.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_epochs_summaries.append(generated_summary)
    references.append(batch['highlights'])

rouge_scores = rouge.get_scores(generated_epochs_summaries, references, avg=True)
print(rouge_scores)

{'rouge-1': {'r': 0.4126912869174646, 'p': 0.30067283870248923, 'f': 0.3396914456172121}, 'rouge-2': {'r': 0.20694422178202626, 'p': 0.14228825302140938, 'f': 0.16532284488394994}, 'rouge-l': {'r': 0.38670389195948146, 'p': 0.2788579161522726, 'f': 0.31633109813373717}}


In [14]:
rouge_scores_finetuned = [rouge.get_scores(gen_summary, ref) for gen_summary, ref in zip(generated_epochs_summaries, references)]

average_rouge1_f_score_finetuned = np.mean([score[0]['rouge-1']['f'] for score in rouge_scores_finetuned])
print("Score ROUGE moyen pour T5 avec entraînement :", average_rouge1_f_score_finetuned)

Score ROUGE moyen pour T5 avec entraînement : 0.33969144561721215


En passant le nombre d'epochs de 3 à 4, le score Rouge a légèrement augmenté. Il est passé de 0,327 à 0,339.

In [12]:
test_dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 14
})

### Sauvegarde des articles et résumés générés dans un fichier CSV

In [ ]:
import pandas as pd


articles = [item['article'] for item in test_dataset]
highlights = [item['highlights'] for item in test_dataset]

# création d'un DataFrame avec les articles et les résumés générés
df = pd.DataFrame({
    'article': articles,
    'highlights': highlights,
    't5_summary': generated_epochs_summaries
})

# sauvegarde dans un fichier CSV
df.to_csv('cnn_daily_t5.csv', index=False)
